##### For all the iterations done on the Actual Data (~ 4000 Independent Features and ~120 rows) we found that the VIF( which runs a Linear Regression multiple times - each time dropping a feature based on VIF/ R-square) : All the first ~ > 3900 features were dropped. Could this be because of number of Independent Features >> number of rows. We then performed the following experiments to validate and conclude.

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yaml

import warnings
warnings.filterwarnings("ignore")

In [2]:
# generate_dataset will create a data with significant features for Linear Regression
def generate_dataset(coeffs, n, std_dev):
    # We calculate the number of predictors, and create a coefficient matrix
    # With `p` rows and 1 column, for matrix multiplication
    p = len(coeffs)
    coeff_mat = np.array(coeffs).reshape(p, 1)
    # Similar as before, but with `n` rows and `p` columns this time
    x = np.random.random_sample((n, p))* 100
    e = np.random.randn(n) * std_dev
    # Since x is a n*p matrix, and coefficients is a p*1 matrix
    # we can use matrix multiplication to get the value of y for each
    # set of values x1, x2 .. xp
    # We need to transpose it to get a 1*n array from a n*1 matrix to use in the regression model
    y = np.matmul(x, coeff_mat).transpose() + e
    return x, y.transpose()

##### Let's test if this function really works: <br /> try this function out: create a data set with 100 Features and 10,000 Observations <br /> Claim: all of the 100 features will be significant

In [3]:
# Calling the yaml file
with open("./Generate_dataset.yml", 'r') as f:
    params = yaml.safe_load(f)

In [4]:
# co-efficients will range from 0. to 50. 
#co_effs1 = list(np.random.choice(50, 100))
# define X: 100 independent features; Y: Dependent Feature
co_effs1 = list(np.random.choice(50, 100))
n= params['n']
std= params['std']
X1, y1 = generate_dataset(co_effs1, n, std)

#### We run the statmodels.sm for Linear Regression <br /> <br /> Print out the output <br /> <br /> and check the p-values (they should all be significant)

In [5]:
%%time
model = sm.OLS(y1, X1).fit()
predictions = model.predict(X1) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.418e+07
Date:                Fri, 31 May 2019   Prob (F-statistic):               0.00
Time:                        16:21:49   Log-Likelihood:                -5217.0
No. Observations:                1000   AIC:                         1.063e+04
Df Residuals:                     900   BIC:                         1.112e+04
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            30.9951      0.053    579.714      0.0

# --------------------------------------------------------------------------

### Let's subset the data ... take 100 observations check p-values

In [6]:
X2, y2 = X1[0:100], y1[0:100]

In [7]:
print(X2.shape, y2.shape)

(100, 100) (100, 1)


In [8]:
%%time
model = sm.OLS(y2, X2).fit()
predictions = model.predict(X1) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Fri, 31 May 2019   Prob (F-statistic):                nan
Time:                        16:21:50   Log-Likelihood:                 2028.3
No. Observations:                 100   AIC:                            -3857.
Df Residuals:                       0   BIC:                            -3596.
Df Model:                          99                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            32.5114        inf          0        n

### --------------------------------------------------------------------

### More experiments <br />  <br />  10,000 Features and 120 Obervations (Random subset from previous data)

In [9]:
idx = np.random.choice(np.arange(len(X1)), 120, replace=False)
X2 = X1[idx]
y2 = y1[idx]

In [10]:
%%time
model2 = sm.OLS(y2, X2).fit()
predictions2 = model2.predict(X2) 

print_model2 = model2.summary()
print(print_model2)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.635e+06
Date:                Fri, 31 May 2019   Prob (F-statistic):           1.99e-64
Time:                        16:21:50   Log-Likelihood:                -531.72
No. Observations:                 120   AIC:                             1263.
Df Residuals:                      20   BIC:                             1542.
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            31.0352      0.350     88.611      0.0